In [1]:
## Import libraries needed for setting parameters of operating system 
import os
import sys

## Import library for temporary files creation 
import tempfile 

## Import Pandas library
import pandas as pd

## Import Numpy library
import numpy

## Import Psycopg2 library (interection with postgres database)
import psycopg2 as pg

# Import Math library (usefull for rounding number, e.g.)
import math

## Import Subprocess + subprocess.call
import subprocess
from subprocess import call, Popen, PIPE, STDOUT


In [2]:
## Define a empty dictionnary for saving user inputs
user={}

## Windows7 Priminfo

## Linux Mint HP

In [3]:
import os,sys
def setup_environmental_variables():
	"""	Setting the environment variables allowing to use of GRASS GIS python libraries 
	Documentaion available on: https://grass.osgeo.org/grass64/manuals/variables.html
	Please change the directory path according to your own system configuration.
	Here after are the path used on a LINUX UBUNTU Mint 18.1 (Serena).
    """
	# Check is environmental variables exists and create them (empty) if not exists.
	if not 'PYTHONPATH' in os.environ:
		os.environ['PYTHONPATH']=''
	if not 'LD_LIBRARY_PATH' in os.environ:
		os.environ['LD_LIBRARY_PATH']=''
	# Set environmental variables
	os.environ['GISBASE'] = config_parameters['GISBASE']
	os.environ['PATH'] += os.pathsep + os.path.join(os.environ['GISBASE'],'bin')
	os.environ['PATH'] += os.pathsep + os.path.join(os.environ['GISBASE'],'script')
	os.environ['PATH'] += os.pathsep + os.path.join(os.environ['GISBASE'],'lib')
	os.environ['PYTHONPATH'] += os.pathsep + os.path.join(os.environ['GISBASE'],'etc','python')
	os.environ['PYTHONPATH'] += os.pathsep + os.path.join(os.environ['GISBASE'],'etc','python','grass')
	os.environ['PYTHONPATH'] += os.pathsep + os.path.join(os.environ['GISBASE'],'etc','python','grass','script')
	os.environ['PYTHONLIB'] = config_parameters['PYTHONLIB']
	os.environ['LD_LIBRARY_PATH'] += os.pathsep + os.path.join(os.environ['GISBASE'],'lib')
	os.environ['GIS_LOCK'] = '$$'
	os.environ['GISRC'] = os.path.join(os.environ['HOME'],'.grass7','rc')  ## Guess will only works for LINUX

	## Define GRASS-Python environment
	sys.path.append(os.path.join(os.environ['GISBASE'],'etc','python'))


def print_environmental_variables():
	"""
	Display the current environmental variables of your computer.
	"""
	## Display the current defined environment variables
	for key in os.environ.keys():
		print ("%s = %s \t" % (key,os.environ[key]))

In [4]:
# Initialize dictionnaries
config_parameters = {}
data = {}

## Please update the following paths according to your own configuration
config_parameters['GISBASE'] = '/usr/lib/grass76'
#config_parameters['PYTHONLIB'] = '/usr/lib/python2.7'
config_parameters['PYTHONLIB'] = '/usr/bin/python2'
config_parameters['njobs'] = 6
# path to GRASSDATA folder
config_parameters['gisdb'] = '/media/tais/data/MAXWELL/GRASSDATA'

In [5]:
user["gisdb"] = config_parameters['gisdb']

In [6]:
# Define output folders for results
outputfolder="/media/tais/data/MAXWELL/Results_2017"
outputfolder_segmentation = os.path.join(outputfolder,'segmentation')
outputfolder_uspo = os.path.join(outputfolder_segmentation,'uspo')

In [7]:
setup_environmental_variables()

In [8]:
print_environmental_variables()

SHELL = /bin/bash 	
SESSION_MANAGER = local/tais-HP-Z620-Workstation:@/tmp/.ICE-unix/1916,unix/tais-HP-Z620-Workstation:/tmp/.ICE-unix/1916 	
WINDOWID = 50331654 	
QT_ACCESSIBILITY = 1 	
COLORTERM = truecolor 	
XDG_CONFIG_DIRS = /etc/xdg/xdg-mate:/etc/xdg 	
XDG_SESSION_PATH = /org/freedesktop/DisplayManager/Session0 	
LANGUAGE = fr_BE:fr 	
MANDATORY_PATH = /usr/share/gconf/mate.mandatory.path 	
SSH_AUTH_SOCK = /run/user/1000/keyring/ssh 	
DESKTOP_SESSION = mate 	
SSH_AGENT_PID = 2175 	
GTK_MODULES = appmenu-gtk-module:gail:atk-bridge:canberra-gtk-module 	
XDG_SEAT = seat0 	
MATE_DESKTOP_SESSION_ID = this-is-deprecated 	
PWD = /home/tais 	
LOGNAME = tais 	
XDG_SESSION_DESKTOP = mate 	
QT_QPA_PLATFORMTHEME = gtk2 	
XDG_SESSION_TYPE = x11 	
GPG_AGENT_INFO = /run/user/1000/gnupg/S.gpg-agent:0:1 	
XAUTHORITY = /home/tais/.Xauthority 	
XDG_GREETER_DATA_DIR = /var/lib/lightdm-data/tais 	
GDM_LANG = fr_FR 	
HOME = /home/tais 	
LANG = fr_BE.UTF-8 	
LS_COLORS = rs=0:di=01;34:ln=01;36:mh=00:pi=40

## Linux Mint HP

In [9]:
## Enter the name of the location (existing or for a new one)
user["location"] = "Accra_32630"
## Enter the EPSG code for this location 
user["locationepsg"] = "32630"
## Enter the name of the mapset to use for Unsupervised Segmentation Parameter Optimization (USPO) step
user["uspo_mapsetname"] = "accra_USPO"
## Enter the name of the mapset to use for segmentation step
user["segmentation_mapsetname"] = "accra_Seg"
## Enter the name of the mapset to use for classification step
user["classification_mapsetname"] = "CLASSIFICATION"

In [10]:
## Import libraries needed to launch GRASS GIS in the jupyter notebook
import grass.script.setup as gsetup

## Import libraries needed to call GRASS using Python
import grass.script as grass

In [11]:
if os.path.exists(user["gisdb"]):
    print ("GRASSDATA folder already exist") 
else: 
    os.makedirs(user["gisdb"]) 
    print ("GRASSDATA folder created in "+user["gisdb"])

GRASSDATA folder already exist


In [12]:
# Automatic creation of GRASS location is doesn't exist
if os.path.exists(os.path.join(user["gisdb"],user["location"])):
    print ("Location "+user["location"]+" already exist") 
else : 
    grass.core.create_location(user["gisdb"], user["location"], epsg=user["locationepsg"], overwrite=False)
    print ("Location "+user["gisdb"]+" created")

Location Accra_32630 already exist


In [13]:
# Automatic creation of GRASS GIS mapsets

## Import library for file copying 
import shutil

## USPO mapset
mapsetname=user["uspo_mapsetname"]
if os.path.exists(os.path.join(user["gisdb"],user["location"],mapsetname)):
    print ("'"+mapsetname+"' mapset already exist") 
else: 
    os.makedirs(os.path.join(user["gisdb"],user["location"],mapsetname))
    shutil.copy(os.path.join(user["gisdb"],user["location"],
                             'PERMANENT','WIND'),os.path.join(user["gisdb"],user["location"],mapsetname,'WIND'))
    print ("'"+mapsetname+"' mapset created in location "+user["gisdb"])

'accra_USPO' mapset already exist


In [14]:
# Automatic creation of GRASS GIS mapsets

## Import library for file copying 
import shutil

## Classification mapset
mapsetname=user["classification_mapsetname"]
if os.path.exists(os.path.join(user["gisdb"],user["location"],mapsetname)):
    print ("'"+mapsetname+"' mapset already exist") 
else: 
    os.makedirs(os.path.join(user["gisdb"],user["location"],mapsetname))
    shutil.copy(os.path.join(user["gisdb"],user["location"],
                             'PERMANENT','WIND'),os.path.join(user["gisdb"],user["location"],mapsetname,'WIND'))
    print ("'"+mapsetname+"' mapset created in location "+user["gisdb"])

'CLASSIFICATION' mapset already exist


In [15]:
## Function which concatenate individual .csv files stored in a folder. 
# 'indir' parameter wait for a string with the path to the repertory to look for individual .csv files
# 'pattern' parameter wait for a string with the pattern of filename to look for (example: "TMP_*.csv")
# 'sep' parameter wait for a string with the delimiter of the .csv file
# 'replacedict' parameter wait for a dictionary containing the unwanted values as keys and the replace string as corresponding value
# 'outputfilename' parameter wait for the name of the outputfile (including extansion but without the complete path)

import os
import glob
import csv

def concat_findreplace(indir,pattern,sep,replacedict,outputfilename):
    # Initialise some variables
    messagetoprint=None
    returnmessage=None
    countdict={}
    for k in replacedict:
        countdict[k]=0
    # Change the working directory
    os.chdir(indir) 
    # Get a list with file in the current directory corresponding to a specific pattern
    fileList=None
    fileList=glob.glob(pattern)
    # Print
    messagetoprint="Going to concatenate "+str(len(fileList))+" .csv files together and replace unwanted values."
    #print (messagetoprint)
    returnmessage=messagetoprint
    # Create a new file 
    outfile=os.path.join(indir, outputfilename)
    writercsvSubset = open(outfile, 'wb')
    writercsv=csv.writer(writercsvSubset,delimiter=sep)
    # Concatenate individuals files and replace unwanted values
    for indivfile in fileList:
        with open(indivfile) as readercsvSubset:
            readercsv=csv.reader(readercsvSubset, delimiter=sep)
            if indivfile!=fileList[0]:
                readercsv.next()
            count=0
            for row in readercsv:
                newline=[]
                for i, x in enumerate(row):
                    if x in replacedict:
                        newline.append(replacedict[x])
                        countdict[x]+=1
                    else:
                        newline.append(row[i])
                writercsv.writerow(newline)
        # Close the current input file
        readercsvSubset.close()
    # Close the outputfile
    writercsvSubset.close()
    # Count number of changes
    countchange=0
    for k in countdict:
        countchange+=countdict[k]
    # Print
    if countchange>0:
        messagetoprint="\n"
        returnmessage+=messagetoprint
        messagetoprint="Values have been changed:"+"\n"
        #print (messagetoprint)
        returnmessage+=messagetoprint
        for k in replacedict:
            if countdict[k]>0:
                messagetoprint=str(countdict[k])+" '"+k+"' value(s) replaced by '"+replacedict[k]+"'\n"
                #print (messagetoprint)
                returnmessage+=messagetoprint
    else:
        messagetoprint="Nothing changed. No unwanted values found !"
        #print (messagetoprint)
        returnmessage+=messagetoprint
    # Return
    return returnmessage[:-1]

In [16]:
## Import library for managing time in python
import time  

## Function "print_processing_time()" compute processing time and printing it.
# The argument "begintime" wait for a variable containing the begintime (result of time.time()) of the process for which to compute processing time.
# The argument "printmessage" wait for a string format with information about the process. 
def print_processing_time(begintime, printmessage):    
    endtime=time.time()           
    processtime=endtime-begintime
    remainingtime=processtime

    days=int((remainingtime)/86400)
    remainingtime-=(days*86400)
    hours=int((remainingtime)/3600)
    remainingtime-=(hours*3600)
    minutes=int((remainingtime)/60)
    remainingtime-=(minutes*60)
    seconds=round((remainingtime)%60,1)

    if processtime<60:
        finalprintmessage=str(printmessage)+str(seconds)+" seconds"
    elif processtime<3600:
        finalprintmessage=str(printmessage)+str(minutes)+" minutes and "+str(seconds)+" seconds"
    elif processtime<86400:
        finalprintmessage=str(printmessage)+str(hours)+" hours and "+str(minutes)+" minutes and "+str(seconds)+" seconds"
    elif processtime>=86400:
        finalprintmessage=str(printmessage)+str(days)+" days, "+str(hours)+" hours and "+str(minutes)+" minutes and "+str(seconds)+" seconds"
    
    return finalprintmessage

# Import data

FileNotFoundError: [Errno 2] No such file or directory: b'r.mask': b'r.mask'

In [23]:
## Saving current time for processing time management
begintime_full=time.time()

In [24]:
## Save the name of the mapset in which to import the data
mapsetname='PERMANENT'

## Launch GRASS GIS working session in the mapset
if os.path.exists(os.path.join(user["gisdb"],user["location"],mapsetname)):
    gsetup.init(os.environ['GISBASE'], user["gisdb"], user["location"], mapsetname)
    print ("You are now working in mapset '"+mapsetname+"'") 
else: 
    print ("'"+mapsetname+"' mapset doesn't exists in "+user["gisdb"])

You are now working in mapset 'PERMANENT'


In [25]:
## Save default computational region to match the full extend of optical imagery
grass.run_command('g.region', flags="s", raster="pan_17@PERMANENT", align="pan_17@PERMANENT")

0

In [41]:
## Check if there is a raster layer named "MASK"
if not grass.list_strings("rast", pattern="MASK", flag='r'):
    print ('There is currently no MASK')
else:
    ## Remove the current MASK layer
    grass.run_command('r.mask',flags='r')
    print ('The current MASK has been removed')

There is currently no MASK


# USPO

In [26]:
## Set the name of the mapset in which to work
mapsetname=user["uspo_mapsetname"]

## Launch GRASS GIS working session in the mapset
if os.path.exists(os.path.join(user["gisdb"],user["location"],mapsetname)):
    gsetup.init(os.environ['GISBASE'], user["gisdb"], user["location"], mapsetname)
    print ("You are now working in mapset '"+mapsetname+"'") 
else: 
    print ("'"+mapsetname+"' mapset doesn't exists in "+user["gisdb"])

You are now working in mapset 'accra_USPO'


## Create grid

In [42]:
grid_size = 500  # border size in meters
min_size_intersected_grid = 5000  # area in square meters
grid_layer = 'grid_%sm'%grid_size

In [43]:
print ("Create grid vector for local USPO at " + time.ctime())
## Define computational region
grass.run_command('g.region', flags="d", align="pan_17@PERMANENT")
#grass.run_command('g.region', flags="d", align="ihs_17_red@PERMANENT")

## Create grid
grass.run_command('v.mkgrid', overwrite=True, map='tmp', box='%s,%s'%(grid_size,grid_size))

# Intersect the grid layer with the extend of the AOI
grass.run_command('g.region', vector='tmp', align="pan_17@PERMANENT")
grass.run_command('v.to.rast', overwrite=True, input='tmp', type="area", output='tmp', use="cat")
grass.run_command('r.mapcalc', overwrite=True,
                  expression="intersected_grid = if( isnull(pan_17),null(),tmp)")
grass.run_command('r.to.vect', overwrite=True, input='intersected_grid', output='tmp_2', type='area')

# Remove very tiny polygons
grass.run_command('v.clean', overwrite=True, input='tmp_2', 
                  output=grid_layer, tool='rmarea', threshold=min_size_intersected_grid)
grass.run_command('v.db.droptable', flags='f', map=grid_layer)
grass.run_command('v.db.addtable', map=grid_layer)

## Add column for area (area_km2)
grass.run_command('v.db.addcolumn', map=grid_layer, columns="area_m2 double precision")
grass.run_command('v.db.addcolumn', map=grid_layer, columns="area_km2 double precision")
grass.run_command('v.to.db', map=grid_layer, option='area', columns='area_m2')
grass.run_command('v.db.update', map=grid_layer, column='area_km2', query_column='area_m2/1000000')
grass.run_command('v.db.dropcolumn', map=grid_layer, columns='area_m2')

# Remove intermediate layers
grass.run_command('g.remove', flags='f', type='vector', name='tmp')
grass.run_command('g.remove', flags='f', type='vector', name='tmp_2')
grass.run_command('g.remove', flags='f', type='raster', name='tmp')

Create grid vector for local USPO at Wed Dec 18 18:06:35 2019


0

**Export grid layer as Shapefile**

In [44]:
if os.path.exists(outputfolder_segmentation):
    print ("folder already exist") 
else: 
    os.makedirs(outputfolder_segmentation) 
    print ("folder created on %s"%outputfolder_segmentation)

folder already exist


In [45]:
output_grid_shape = os.path.join(outputfolder_segmentation,'grid_uspo.shp')
grass.run_command('v.out.ogr', overwrite=True, input=grid_layer, output=output_grid_shape, format='ESRI_Shapefile')
print ("Grid layer successfully exported")

Grid layer successfully exported


## Create group of layers of i.segment.uspo

In [31]:
# Define name of group of image for uspo
uspo_group = 'group'

In [32]:
print ("Defining a imagery group with raster used for i.segment.uspo at " + time.ctime())

## Remove existing imagery group named "group"
grass.run_command('g.remove', flags="rf", type="group", name=uspo_group)
 
## Add all optical imagery in the imagery group
list_rast = grass.list_strings("rast", pattern="pan_17", flag='r')
#list_rast = grass.list_strings("rast", pattern="ihs_17", flag='r')

grass.run_command('i.group', group=uspo_group, input="pan_17")
#grass.run_command('i.group', group=uspo_group, input=','.join(list_rast))

## Add NDVI imagery in the imagery group
#grass.run_command('i.group', group=uspo_group, input="NDVI@PERMANENT")

## list files in the group
print (grass.read_command('i.group', group=uspo_group, flags="l"))

Defining a imagery group with raster used for i.segment.uspo at Wed Dec 18 18:02:47 2019
b'Le groupe <group> r\xc3\xa9f\xc3\xa9rence les cartes raster suivantes\n-------------\n<pan_17@PERMANENT>    \n-------------\n'


## Local USPO segmentation

In [33]:
if os.path.exists(outputfolder_uspo):
    print ("folder already exist") 
else: 
    os.makedirs(outputfolder_uspo) 
    print ("folder created on %s"%outputfolder_uspo)

folder already exist


### Define optimization parameter

In [53]:
## Define the optimization function name ("sum" or "f")
opti_f="f"

## Define the alpha, only if selected optimization function is "f"
if opti_f=="f":
    alpha=1
else : alpha=""
    
## Define the threshold range and step 
thr_start = 0.005
thr_stop = 0.04
thr_step = 0.002

## Define the minsize (single value)
minsize = 5

In [54]:
## Save the list of polygons to be processed (save the 'cat' value)
listofregion=list(grass.parse_command('v.db.select', map=grid_layer, columns='cat', flags='c'))[:]
count=1
for x in listofregion:
    #print str(count)+"      "+str(x)
    count+=1

In [55]:
"prints the list of regions"
#listofregion
listofregion[:]

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '23',
 '24',
 '25',
 '26',
 '27',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '84',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138',
 '139',
 '140',
 '141',
 '142',


In [57]:
## Initialize a empty string for saving print outputs
txtcontent=""

## Running i.segment.uspo
messagetoprint="Runing i.segment.uspo at " + time.ctime()
print (messagetoprint)
txtcontent+=messagetoprint+"\n"

## Compute total area to be segmented for process progression information
nbrtile=len(listofregion)
total_area = float(grass.parse_command('v.db.univar', flags='g', map=grid_layer, column='area_km2')['sum'])
processed_area=0
messagetoprint=str(nbrtile)+" region(s) will be processed, covering an area of "+str(round(total_area,3))+" Sqkm."+"\n\n"
print (messagetoprint)
txtcontent+=messagetoprint

## Save time at beginning of USPO process
begintime_USPO=time.time()

## Start loop on morphological zones
count = 1
error_cat = []
for cat in listofregion[:]:
#for cat in ['2080', '2081']:
    try:
        ## Save current time at loop' start. 
        begintime_current_id=time.time()

        ## Create a computional region for the current polygon
        condition="cat="+cat
        outputname="tmp_"+cat
        regionname="uspo_zm_"+cat
        grass.run_command('v.extract', overwrite=True, quiet=True, 
                          input=grid_layer, type="area", where=condition, output=outputname)
        grass.run_command('g.region', overwrite=True, vector=outputname, save=regionname, 
                          align="pan_17@PERMANENT")
        grass.run_command('r.mask', overwrite=True, vector=outputname)
        grass.run_command('g.remove', quiet=True, type="vector", name=outputname, flags="f")

        ## Save size of the current polygon and add it to the already processed area 
        size=float(grass.read_command('v.db.select', map=grid_layer, columns="area_km2", where=condition,flags="c"))

        ## Define the csv output file name, according to the optimization function selected
        outputcsv=os.path.join(outputfolder_uspo,"uspo_"+str(opti_f)+str(alpha)+regionname[regionname.index("zm"):]+".csv")
        ## Create a .csvt file containing each colomn type of i.segment.uspo' csv output. Required for further import of .csv file
        model_output_desc = outputcsv + "t"
        f = open(model_output_desc, 'w')
        header_string = '"String","Real","Integer","Real","Real","Real"'
        f.write(header_string)
        f.close()

        ## Print
        messagetoprint="Starting segmentation of morphological zone "+str(cat)+" (n°"+str(count)+"/"+str(nbrtile)+")"
        messagetoprint+=" covering "+str(round(size,3))+" Sqkm."
        print (messagetoprint)
        txtcontent+=messagetoprint+"\n"

        ## i.segment.uspo
        grass.run_command('i.segment.uspo', overwrite=True, group=uspo_group, 
                      output=outputcsv, segment_map="best", regions=regionname, 
                      threshold_start=thr_start, threshold_stop=thr_stop, threshold_step=thr_step, minsizes=minsize, 
                      optimization_function=opti_f, f_function_alpha=alpha, memory="40000", processes="25")

        ## Cleaning the output .csv file of i.segment.uspo to get the first ranked segmentation in a new csv
        df=pd.read_csv(outputcsv, sep=',',header=0)
        temp=df.loc[:,['region','optimization_criteria']].groupby('region').max()
        temp['optimization_criteria'] = temp ['optimization_criteria'].astype (float)
        df['optimization_criteria'] = df ['optimization_criteria'].astype (float)
        ## Merge between dataframes for identification of threshold corresponding to the maximum optimizaion criteria of each "USPO's region"
        uspo_parameters = pd.merge(df, temp, on='optimization_criteria').sort_values(by='region')
        ## Rename USPO region to match the "type" column of "zone_morpho" layer
        currentvalue=uspo_parameters.loc[0:0,['region']].values[0][0]
        pattern="uspo_zm_"
        beginindex=currentvalue.index(pattern)+len(pattern)
        newvalue=currentvalue[beginindex:]
        uspo_parameters.loc[0:0,['region']]=newvalue
        uspo_parameters['area']=size

        ## Define the csv output file name with the best segmentation parameters
        outputbest=os.path.join(outputfolder_uspo,"best_uspo_"+str(opti_f)+str(alpha)+"_"+regionname[regionname.index("zm"):]+".csv")
        uspo_parameters.to_csv(path_or_buf=outputbest ,sep=',', header=True,  quoting=None, decimal='.', index=False)

        ## Save the threshold used for the current zone
        thresh=uspo_parameters['threshold'][0]
        ## Add the size of the zone to the already processed area
        processed_area+=size

        ## Print
        messagetoprint="The current zone was segmented with threshold:"+str(thresh)
        print (messagetoprint)
        txtcontent+=messagetoprint+"\n"
        messagetoprint=print_processing_time(begintime_current_id, "Zone segmented in ")
        print (messagetoprint)
        txtcontent+=messagetoprint+"\n"
        remainingtile=nbrtile-count
        if remainingtile>0:
            messagetoprint=str(round((processed_area/total_area)*100,2))+" percent of the total area processed. "
            messagetoprint+="Still "+str(remainingtile)+" zone(s) to process."+"\n"
            print (messagetoprint)
            txtcontent+=messagetoprint+"\n"
        else:
            messagetoprint="\n"
            print (messagetoprint)
            txtcontent+=messagetoprint

        ## Remove current mask
        grass.run_command('r.mask', flags='r')

        ## Adapt the count 
        count+=1
    except:
        print("An error occured for grid with cat = %s."%cat)
        error_cat.append(cat)

## Compute processing time and print it
messagetoprint=print_processing_time(begintime_USPO, "Segmentation process achieved in ")
print (messagetoprint) 
txtcontent+=messagetoprint

## Inform about potential error
if len(error_cat) > 0:    
    messagetoprint="An error occured for these tiles: %s. Please check for fix and reprocess them again."%','.join(error_cat).encode('utf-8')
    print (messagetoprint) 
    txtcontent+=messagetoprint

#### Write text file with log of processing time
## Create the .txt file for processing time output and begin to write
f = open(os.path.join(outputfolder,mapsetname+"_processingtime_isegmentuspo.txt"), 'w')
f.write(mapsetname+" processing time information for i.segment.uspo"+"\n\n")
f.write(txtcontent)
f.close()

Runing i.segment.uspo at Wed Dec 18 18:21:47 2019
3077 region(s) will be processed, covering an area of 733.306 Sqkm.


Starting segmentation of morphological zone 1 (n°1/3077) covering 0.248 Sqkm.
The current zone was segmented with threshold:0.025
Zone segmented in 4.8 seconds
0.03 percent of the total area processed. Still 3076 zone(s) to process.

Starting segmentation of morphological zone 2 (n°2/3077) covering 0.25 Sqkm.
The current zone was segmented with threshold:0.025
Zone segmented in 4.7 seconds
0.07 percent of the total area processed. Still 3075 zone(s) to process.

Starting segmentation of morphological zone 3 (n°3/3077) covering 0.25 Sqkm.
The current zone was segmented with threshold:0.027000000000000003
Zone segmented in 4.7 seconds
0.1 percent of the total area processed. Still 3074 zone(s) to process.

Starting segmentation of morphological zone 4 (n°4/3077) covering 0.25 Sqkm.
The current zone was segmented with threshold:0.023
Zone segmented in 4.9 seconds
0.14 pe

Starting segmentation of morphological zone 37 (n°35/3077) covering 0.056 Sqkm.
The current zone was segmented with threshold:0.025
Zone segmented in 3.0 seconds
0.83 percent of the total area processed. Still 3042 zone(s) to process.

Starting segmentation of morphological zone 38 (n°36/3077) covering 0.25 Sqkm.
The current zone was segmented with threshold:0.027000000000000003
Zone segmented in 4.7 seconds
0.86 percent of the total area processed. Still 3041 zone(s) to process.

Starting segmentation of morphological zone 39 (n°37/3077) covering 0.25 Sqkm.
The current zone was segmented with threshold:0.025
Zone segmented in 4.8 seconds
0.89 percent of the total area processed. Still 3040 zone(s) to process.

Starting segmentation of morphological zone 40 (n°38/3077) covering 0.25 Sqkm.
The current zone was segmented with threshold:0.027000000000000003
Zone segmented in 5.3 seconds
0.93 percent of the total area processed. Still 3039 zone(s) to process.

Starting segmentation of morp

Starting segmentation of morphological zone 71 (n°69/3077) covering 0.124 Sqkm.
The current zone was segmented with threshold:0.027000000000000003
Zone segmented in 3.6 seconds
1.84 percent of the total area processed. Still 3008 zone(s) to process.

Starting segmentation of morphological zone 72 (n°70/3077) covering 0.115 Sqkm.
The current zone was segmented with threshold:0.027000000000000003
Zone segmented in 3.5 seconds
1.86 percent of the total area processed. Still 3007 zone(s) to process.

Starting segmentation of morphological zone 73 (n°71/3077) covering 0.105 Sqkm.
The current zone was segmented with threshold:0.025
Zone segmented in 3.4 seconds
1.87 percent of the total area processed. Still 3006 zone(s) to process.

Starting segmentation of morphological zone 74 (n°72/3077) covering 0.096 Sqkm.
The current zone was segmented with threshold:0.025
Zone segmented in 3.4 seconds
1.88 percent of the total area processed. Still 3005 zone(s) to process.

Starting segmentation of m

Starting segmentation of morphological zone 107 (n°103/3077) covering 0.25 Sqkm.
The current zone was segmented with threshold:0.027000000000000003
Zone segmented in 4.7 seconds
2.7 percent of the total area processed. Still 2974 zone(s) to process.

Starting segmentation of morphological zone 108 (n°104/3077) covering 0.251 Sqkm.
The current zone was segmented with threshold:0.027000000000000003
Zone segmented in 4.7 seconds
2.73 percent of the total area processed. Still 2973 zone(s) to process.

Starting segmentation of morphological zone 109 (n°105/3077) covering 0.25 Sqkm.
The current zone was segmented with threshold:0.025
Zone segmented in 4.8 seconds
2.76 percent of the total area processed. Still 2972 zone(s) to process.

Starting segmentation of morphological zone 110 (n°106/3077) covering 0.25 Sqkm.
The current zone was segmented with threshold:0.025
Zone segmented in 4.9 seconds
2.8 percent of the total area processed. Still 2971 zone(s) to process.

Starting segmentation o

Starting segmentation of morphological zone 141 (n°137/3077) covering 0.251 Sqkm.
The current zone was segmented with threshold:0.025
Zone segmented in 4.8 seconds
3.83 percent of the total area processed. Still 2940 zone(s) to process.

Starting segmentation of morphological zone 142 (n°138/3077) covering 0.25 Sqkm.
The current zone was segmented with threshold:0.025
Zone segmented in 4.9 seconds
3.86 percent of the total area processed. Still 2939 zone(s) to process.

Starting segmentation of morphological zone 143 (n°139/3077) covering 0.208 Sqkm.
The current zone was segmented with threshold:0.023
Zone segmented in 4.4 seconds
3.89 percent of the total area processed. Still 2938 zone(s) to process.

Starting segmentation of morphological zone 144 (n°140/3077) covering 0.237 Sqkm.
The current zone was segmented with threshold:0.025
Zone segmented in 4.7 seconds
3.92 percent of the total area processed. Still 2937 zone(s) to process.

Starting segmentation of morphological zone 145 (

...

The current zone was segmented with threshold:0.025
Zone segmented in 3.0 seconds
95.43 percent of the total area processed. Still 154 zone(s) to process.

Starting segmentation of morphological zone 2943 (n°2924/3077) covering 0.18 Sqkm.
The current zone was segmented with threshold:0.027000000000000003
Zone segmented in 4.2 seconds
95.45 percent of the total area processed. Still 153 zone(s) to process.

Starting segmentation of morphological zone 2944 (n°2925/3077) covering 0.25 Sqkm.
The current zone was segmented with threshold:0.025
Zone segmented in 4.8 seconds
95.49 percent of the total area processed. Still 152 zone(s) to process.

Starting segmentation of morphological zone 2945 (n°2926/3077) covering 0.25 Sqkm.
The current zone was segmented with threshold:0.025
Zone segmented in 4.7 seconds
95.52 percent of the total area processed. Still 151 zone(s) to process.

Starting segmentation of morphological zone 2946 (n°2927/3077) covering 0.25 Sqkm.
The current zone was segmente

The current zone was segmented with threshold:0.025
Zone segmented in 4.8 seconds
96.51 percent of the total area processed. Still 120 zone(s) to process.

Starting segmentation of morphological zone 2978 (n°2958/3077) covering 0.25 Sqkm.
The current zone was segmented with threshold:0.023
Zone segmented in 5.0 seconds
96.54 percent of the total area processed. Still 119 zone(s) to process.

Starting segmentation of morphological zone 2979 (n°2959/3077) covering 0.25 Sqkm.
The current zone was segmented with threshold:0.025
Zone segmented in 4.7 seconds
96.57 percent of the total area processed. Still 118 zone(s) to process.

Starting segmentation of morphological zone 2980 (n°2960/3077) covering 0.25 Sqkm.
The current zone was segmented with threshold:0.027000000000000003
Zone segmented in 4.9 seconds
96.61 percent of the total area processed. Still 117 zone(s) to process.

Starting segmentation of morphological zone 2981 (n°2961/3077) covering 0.25 Sqkm.
The current zone was segmente

Starting segmentation of morphological zone 3011 (n°2991/3077) covering 0.25 Sqkm.
The current zone was segmented with threshold:0.023
Zone segmented in 4.9 seconds
97.56 percent of the total area processed. Still 86 zone(s) to process.

Starting segmentation of morphological zone 3012 (n°2992/3077) covering 0.25 Sqkm.
The current zone was segmented with threshold:0.025
Zone segmented in 4.7 seconds
97.6 percent of the total area processed. Still 85 zone(s) to process.

Starting segmentation of morphological zone 3013 (n°2993/3077) covering 0.251 Sqkm.
The current zone was segmented with threshold:0.025
Zone segmented in 5.0 seconds
97.63 percent of the total area processed. Still 84 zone(s) to process.

Starting segmentation of morphological zone 3014 (n°2994/3077) covering 0.25 Sqkm.
The current zone was segmented with threshold:0.027000000000000003
Zone segmented in 5.0 seconds
97.66 percent of the total area processed. Still 83 zone(s) to process.

Starting segmentation of morpholo

The current zone was segmented with threshold:0.027000000000000003
Zone segmented in 4.9 seconds
98.54 percent of the total area processed. Still 53 zone(s) to process.

Starting segmentation of morphological zone 3045 (n°3025/3077) covering 0.25 Sqkm.
The current zone was segmented with threshold:0.025
Zone segmented in 4.8 seconds
98.57 percent of the total area processed. Still 52 zone(s) to process.

Starting segmentation of morphological zone 3046 (n°3026/3077) covering 0.25 Sqkm.
The current zone was segmented with threshold:0.023
Zone segmented in 4.9 seconds
98.61 percent of the total area processed. Still 51 zone(s) to process.

Starting segmentation of morphological zone 3047 (n°3027/3077) covering 0.25 Sqkm.
The current zone was segmented with threshold:0.023
Zone segmented in 5.2 seconds
98.64 percent of the total area processed. Still 50 zone(s) to process.

Starting segmentation of morphological zone 3048 (n°3028/3077) covering 0.25 Sqkm.
The current zone was segmented wi

Starting segmentation of morphological zone 3080 (n°3058/3077) covering 0.089 Sqkm.
The current zone was segmented with threshold:0.023
Zone segmented in 3.5 seconds
99.53 percent of the total area processed. Still 19 zone(s) to process.

Starting segmentation of morphological zone 3081 (n°3059/3077) covering 0.24 Sqkm.
The current zone was segmented with threshold:0.027000000000000003
Zone segmented in 4.5 seconds
99.56 percent of the total area processed. Still 18 zone(s) to process.

Starting segmentation of morphological zone 3082 (n°3060/3077) covering 0.25 Sqkm.
The current zone was segmented with threshold:0.027000000000000003
Zone segmented in 4.7 seconds
99.6 percent of the total area processed. Still 17 zone(s) to process.

Starting segmentation of morphological zone 3083 (n°3061/3077) covering 0.25 Sqkm.
The current zone was segmented with threshold:0.027000000000000003
Zone segmented in 4.7 seconds
99.63 percent of the total area processed. Still 16 zone(s) to process.

Sta

## Merging all uspo results in one single dataframe

In [58]:
## Initialize a empty list for saving .csv outputs from i.segment.uspo in dataframe
dflist=[]

for cat in listofregion[:]:
    ## Define the csv output file name, according to the optimization function selected
    outputbest=os.path.join(outputfolder_uspo,"best_uspo_"+str(opti_f)+str(alpha)+"_zm_"+str(cat))+".csv"
    ## Open the .csv file of i.segment.uspo to get the first ranked segmentation in a new csv
    df=pd.read_csv(outputbest, sep=',',header=0)  
    ## Save the dataframe in a list
    dflist.append(df)

In [59]:
## Concatenate the dataframes in a single one
resultdf=pd.concat(dflist)
cols = resultdf.columns.tolist()
cols = cols[:1] + cols[-2:] + cols[1:-2]
resultdf = resultdf[cols]

## Export the dataframe as a .csv file
resultcsv=os.path.join(outputfolder_uspo,mapsetname+"_uspo_results_"+str(opti_f)+str(alpha)+".csv")
resultdf.to_csv(path_or_buf=resultcsv ,sep=',', header=True,  quoting=None, decimal='.', index=False)

In [60]:
## Count number of lines in the dataframe
print (str(len(resultdf))+" zones in the dataframes")

3077 zones in the dataframes


In [61]:
resultdf.head()

,region,optimization_criteria,area,threshold,minsize,variance,spatial_autocorrelation
0,1,0.649376,0.248006,0.025,5,31.039610,0.467375
0,2,0.625928,0.250250,0.025,5,39.006765,0.512882
0,3,0.656627,0.249500,0.027,5,38.177297,0.423048
0,4,0.725634,0.249500,0.023,5,32.654301,0.592878
0,5,0.663604,0.250250,0.023,5,35.635283,0.646494


## Merging segmentations in one "patched" raster


The different segmentation results of 'i.segment' (raster layers) for each processing tiles will be "patched" (merged) in one resulting raster. 'r.mapcalc' command is used to combine all the segmentation rasters together. The 'nmax' expression is used to keep the maximum value of input rasters, excluding the NULL values.

In [65]:
## Setting the computational region extend to all rasters to be merged 
groupraster=[]
i=0
for rast in [str(x, 'utf-8') for x in grass.list_strings("rast", pattern="best_uspo_zm", mapset=mapsetname, flag='r')][:]:
    groupraster.append(rast)
    i+=1
print (str(len(groupraster))+" raster found")

3077 raster found


In [66]:
# Create a VRT with all individual segmentation results
grass.run_command('r.buildvrt', overwrite=True, input=','.join(groupraster), output='unclumped_segment_raster')

0

# Clump patched raster

We use here the 'r.clump' command to allow a new (unique) ID for each group of pixels with different values from their neighbors (because segments resulting from 'i.segment' on different processing tiles could have the same ID after being patched in the precedent step).

In [67]:
## Initialize a empty string for saving print outputs
txtcontent=""

## Print and saving current time for processing time management
messagetoprint="Begin clump of segmentation raster on " + time.ctime()+"\n"
print (messagetoprint)
txtcontent+=messagetoprint
begintime_clump=time.time()

## Define the computational region on the whole set of individual segmentation results 
grass.run_command('g.region', overwrite=True, raster=groupraster)

## Generate new individual values for group of pixels
grass.run_command('r.clump', overwrite=True, input="unclumped_segment_raster@"+mapsetname, 
                  output="segmentation_raster@"+mapsetname)

## Compute processing time and print it
messagetoprint=print_processing_time(begintime_clump, "Segment raster have been clumped in ")
print (messagetoprint)
txtcontent+=messagetoprint

## Compute basic statistics about the clumped raster. The maximum value correspond to the number of objets (patchs)
nbrobject=grass.raster_info("segmentation_raster")
messagetoprint="The segment raster contains "+str(int(nbrobject.max))+" objects"+"\n\n"
print(messagetoprint)
txtcontent+=messagetoprint

#### Write text file with log of processing time
## Create the .txt file for processing time output and begin to write
f = open(os.path.join(outputfolder,mapsetname+"_processingtime_clumpsegment.txt"), 'w')
f.write(mapsetname+" processing time information for clumping segmentation_raster with r.clump"+"\n\n")
f.write(txtcontent)
f.close()

Begin clump of segmentation raster on Thu Dec 19 11:09:28 2019

Segment raster have been clumped in 14 minutes and 14.4 seconds
The segment raster contains 16668874 objects




# Export segmentation as .tif image

In [69]:
## Set the region to match the extend of the layer
grass.run_command('g.region', raster='segmentation_raster')
## Export the raster layer as tif image
grass.run_command('r.out.gdal', overwrite=True, input='segmentation_raster', 
                  output=os.path.join(outputfolder_segmentation,mapsetname+"segmentation_raster.tif"),
                  format='GTiff', createopt='COMPRESS=DEFLATE')

0

# Erase intermediate maps

The next cell will remove all the temporary files needed for the different previous steps. Be careful to be sure that your 'segmentation_raster' has correctly been processed before running this part, otherwise you should start all the segmentation part of this script again

In [70]:
## Print 
print ("Begin deleting temporary maps on " + time.ctime())

## Delete individual segmentation rasters 
grass.run_command('g.remove', flags="rf", type="raster", pattern="best_uspo_zm")

## Delete unclumped segmentation rasters 
grass.run_command('g.remove', flags="rf", type="raster", pattern="unclumped_segment_raster")

Begin deleting temporary maps on Thu Dec 19 13:05:58 2019


0